## Virtual Environment for Individual-Based Modeling

This virtual environment is based on an Agent-Based Modeling system using some assumptions that are derived from previousobservations of waterbirds’ behaviors within some habitats in the tropics.

Created on March 31, 2019 by Ralph Florent <r.florent@jacobs-university.de>

In [1]:
# Virtual Environment for Individual-Based Modeling (IBM)
#
# Created on March 31, 2019
#
# Authors:
#   Ralph Florent <r.florent@jacobs-university.de>
#   Davi Tavares <davi.tavares@leibniz-zmt.de>
#   Agostino Merico <a.merico@jacobs-university.de>
#
# Main entry point for the application

# ==============================================================================
# START: Preamble
# ==============================================================================
%reset

# -*- coding: utf-8 -*-
import config
import constants as CONST
from core import initialize, observe, update
from helpers import plot_figure, make_gif

# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
# reload external python modules
%load_ext autoreload
%autoreload 2
%reload_ext autoreload


# ==============================================================================
# END: Preamble
# ==============================================================================

# ==============================================================================
# START: Application
# ==============================================================================

# main entry point for the application
# TODO: proper docs
def application():
    
    # pre-conditions
    config.init() # initialize internal config for the app
    time = 0 # define stopwatch for the process
    
     # process for t times
    print('=> START: Processing random movements based on prob dist')
    habitats, agents = initialize()
    observe(habitats, agents, time)
    
    for time in range(1, CONST.PROCESSING_TIME):
        agents = update(habitats, agents, time) # override agents when being updated
        observe(habitats, agents, time)
    
    print('=> END: Processing random movements based on prob dist')
    
    # post-conditions
    make_gif(CONST.SAMPLE_DIR, 'image.gif', CONST.STORE['images'])
    plot_figure()

    
# run application
application()

# ==============================================================================
# END: Application
# ==============================================================================

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
=> START: Processing random movements based on prob dist
--- snapshot stats: {'lagoon-orange-sm': {'short-legged': 7, 'long-legged': 4}, 'lagoon-orange-lg': {'short-legged': 13, 'long-legged': 5}, 'lagoon-blue': {'short-legged': 0, 'long-legged': 7}, 'lagoon-green': {'short-legged': 0, 'long-legged': 4}}
--- process update: 1
--- snapshot stats: {'lagoon-orange-sm': {'short-legged': 9, 'long-legged': 3}, 'lagoon-orange-lg': {'short-legged': 11, 'long-legged': 10}, 'lagoon-blue': {'short-legged': 0, 'long-legged': 2}, 'lagoon-green': {'short-legged': 0, 'long-legged': 5}}
--- process update: 2
--- snapshot stats: {'lagoon-orange-sm': {'short-legged': 7, 'long-legged': 6}, 'lagoon-orange-lg': {'short-legged': 13, 'long-legged': 10}, 'lagoon-blue': {'short-legged': 0, 'long-legged': 0}, 'lagoon-green': {'short-legged': 0, 'long-legged': 4}}
--- process update: 3
--- snapshot stats: {'lagoon-orange-sm': {'short-legged': 6, 'lo

<Figure size 1100x650 with 0 Axes>

<Figure size 1100x650 with 4 Axes>